Based on the [Finding Detection Mistakes with FiftyOne Tuturial](https://docs.voxel51.com/tutorials/detection_mistakes.html).

In [1]:
import fiftyone as fo
from fiftyone import ViewField as F
from icecream import ic


In [2]:
dataset = fo.load_dataset('dataset5')

In [3]:
print(dataset)

Name:        dataset5
Media type:  image
Num samples: 59997
Persistent:  True
Tags:        []
Sample fields:
    id:                 fiftyone.core.fields.ObjectIdField
    filepath:           fiftyone.core.fields.StringField
    tags:               fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:         fiftyone.core.fields.DateTimeField
    last_modified_at:   fiftyone.core.fields.DateTimeField
    ground_truth:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    YOLOv8_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [6]:
# Open the dataset in the App
fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


Dataset:          dataset5
Media type:       image
Num samples:      59997
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

In [ ]:
patches_view = dataset.to_patches("ground_truth")
print(patches_view)

Dataset:          dataset5
Media type:       image
Num samples:      59997
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/

In [9]:
import fiftyone.brain as fob

# Compute mistakenness of annotations in `ground_truth` field using
# predictions from `predictions` field as point of reference
fob.compute_mistakenness(dataset, "YOLOv8_predictions", label_field="ground_truth")

Evaluating detections...
 100% |█████████████| 59997/59997 [2.1m elapsed, 0s remaining, 540.3 samples/s]      
Computing mistakenness...
 100% |█████████████| 59997/59997 [1.9m elapsed, 0s remaining, 571.7 samples/s]      
Mistakenness computation complete


In [10]:
from fiftyone import ViewField as F

# Sort by likelihood of mistake (most likely first)
mistake_view = dataset.sort_by("mistakenness", reverse=True)

# Print some information about the view
print(mistake_view)

Dataset:     dataset5
Media type:  image
Num samples: 59997
Sample fields:
    id:                 fiftyone.core.fields.ObjectIdField
    filepath:           fiftyone.core.fields.StringField
    tags:               fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:         fiftyone.core.fields.DateTimeField
    last_modified_at:   fiftyone.core.fields.DateTimeField
    ground_truth:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    YOLOv8_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    mistakenness:       fiftyone.core.fields.FloatField
    possible_missing:   fiftyone.core.fields.IntField
    possible_spurious:  fiftyone.core.fields.IntField
View stages:
    1. SortBy(field_or_expr='mistakenness', reverse=True, create_index=True)


In [11]:
# Inspect some samples and detections
# This is the first detection of the first sample
print(mistake_view.first().ground_truth.detections[0])

<Detection: {
    'id': '6736b58b7698ad55d4ec4ca9',
    'attributes': {},
    'tags': [],
    'label': 'live',
    'bounding_box': [0.041749999999999995, 0.69385, 0.0917, 0.1305],
    'mask': None,
    'confidence': None,
    'index': None,
    'mistakenness': 0.052697449922561646,
    'mistakenness_loc': 0.09355670594444199,
}>


In [12]:
# Show the samples we processed in rank order by the mistakenness
session.view = mistake_view

In [ ]:
# Get list of ground truth detection ids tagged with "remove"

removal_list = []
for sample in dataset:
    for detection in sample.ground_truth.detections:
        if 'remove' in detection.tags:   
            removal_list.append(detection.id)
ic(len(removal_list))

ic| len(removal_list): 40


40

In [2]:
dataset = fo.load_dataset('dataset5')
session = fo.launch_app(dataset, auto=False)

# Sort by likelihood of mistake (most likely first)
mistake_view = dataset.sort_by("mistakenness", reverse=True)

# Print some information about the view
print(mistake_view)

# session.view = dataset.filter_labels("ground_truth", F("mistakenness") > 0.70)

Session launched. Run `session.show()` to open the App in a cell output.
Dataset:     dataset5
Media type:  image
Num samples: 59997
Sample fields:
    id:                 fiftyone.core.fields.ObjectIdField
    filepath:           fiftyone.core.fields.StringField
    tags:               fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:           fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:         fiftyone.core.fields.DateTimeField
    last_modified_at:   fiftyone.core.fields.DateTimeField
    ground_truth:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    YOLOv8_predictions: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    mistakenness:       fiftyone.core.fields.FloatField
    possible_missing:   fiftyone.core.fields.IntField
    possible_spurious:  fiftyone.core.fields.IntField
View stages:
    1. SortBy(field_or_expr='mistakenness', rev

In [6]:
mistake_view = dataset.sort_by("mistakenness", reverse=True)
session.view = mistake_view